# Text embeddings

Celem niniejszego zadania jest porównanie jakości i właściwości różnych metod reprezentacji tekstu w postaci wektorowej.
W ramach ćwiczenia zostaną wygenerowane osadzenia zdań (tweetów) przy użyciu dwóch różnych modeli — klasycznych oraz opartych na sieciach neuronowych:


1.   Word2Vec - model oparty na wektorach słów, w którym reprezentacja zdania powstaje poprzez uśrednienie wektorów poszczególnych słów,
2.   Universal Sentence Encoder (USE) - model neuronowy kodujący całe zdania w jeden spójny wektor o stałym wymiarze, odzwierciedlający ich znaczenie semantyczne.

Dla każdego z modeli zostanie wykonana:

1. redukcja wymiarów metodą t-SNE, pozwalająca zwizualizować zależności między osadzeniami,

2. interaktywna wizualizacja, w której możliwe będzie podejrzenie treści tweeta i przypisanej anotacji po najechaniu kursorem na punkt.

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.manifold import TSNE
from tqdm import tqdm
import numpy as np
import pandas as pd
import re
import plotly.express as px
import fasttext
import os, shutil
from numpy.linalg import norm
from itertools import combinations
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
DRIVE_DIR = "/content/drive/MyDrive/Magisterka AI PWR/2 semestr/02 Przetwarzanie języka naturalnego"

# Tweets import with annotations

In [21]:
DRIVE_CSV = f"{DRIVE_DIR}/tweets_pl.csv"

In [22]:
!cp "$DRIVE_CSV" /content/

In [23]:
tweets = pd.read_csv("tweets_pl.csv", sep=';')

In [24]:
tweets

,tweet,annotation
0,Nikt nigdy nie rozsiewał takiego smrodu jak @a...,1
1,Nikt cię tak dobrze nie ubierze jak matka chrz...,0
2,@anonymized_account @anonymized_account Nikt t...,0
3,@anonymized_account A ksiądz co bierze 12 tyś ...,0
4,@anonymized_account Koń też ma dużą głowę. Tyl...,1
...,...,...
96,@anonymized_account Ty zagrasz? Nie wiedziałem 😉,0
97,@anonymized_account @anonymized_account A VAR ...,0
98,@anonymized_account @anonymized_account Szanow...,0
99,@anonymized_account @anonymized_account @anony...,0


In [25]:
tweets["tweet"] = tweets["tweet"].str.replace(r"@anonymized_account", "", regex=True, case=False)
tweets["tweet"] = tweets["tweet"].str.strip()
tweets.head()

,tweet,annotation
0,Nikt nigdy nie rozsiewał takiego smrodu jak,1
1,Nikt cię tak dobrze nie ubierze jak matka chrz...,0
2,Nikt tutaj nie chce plagiaciarzy kopiujących b...,0
3,A ksiądz co bierze 12 tyś za spowiedź przez tel?,0
4,"Koń też ma dużą głowę. Tylko wydaje mi się, że...",1


# FastText

In [7]:
!pip -q install fasttext==0.9.2 || pip -q install fasttext-wheel==0.9.2

In [8]:
LOCAL_BIN = "/content/cc.pl.300.bin"
DRIVE_BIN = f"{DRIVE_DIR}/cc.pl.300.bin"

if os.path.exists(DRIVE_BIN):
    !cp "$DRIVE_BIN" /content/
else:
    !wget -q https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.pl.300.bin.gz
    !gunzip -f cc.pl.300.bin.gz
    !cp "/content/cc.pl.300.bin" "$DRIVE_DIR"

In [ ]:
ft_model = fasttext.load_model(LOCAL_BIN)
TOKENIZER = re.compile(r"\w+", re.UNICODE)

def normalize(text: str):
    text = re.sub(r"http\S+|www\.\S+", " ", str(text))
    text = re.sub(r"\s+", " ", text).strip()
    return text

def embed_sentence(text: str) -> np.ndarray:
    return ft_model.get_sentence_vector(text)

tqdm.pandas()
FT_embeddings = np.vstack(
    tweets["tweet"].astype(str).progress_apply(lambda t: embed_sentence(normalize(t))).values
)
FT_embeddings.shape

100%|██████████| 101/101 [00:00<00:00, 8730.57it/s]


(101, 300)

In [11]:
np.save("/content/embeddings_fasttext.npy", FT_embeddings)

df_fasttext = pd.concat([tweets.reset_index(drop=True),pd.DataFrame(FT_embeddings, columns=[f"ft_{i}" for i in range(FT_embeddings.shape[1])])],
    axis=1
)

df_fasttext.to_csv("/content/tweets_with_fasttext_embeddings.csv", index=False)

!cp "/content/embeddings_fasttext.npy" "$DRIVE_DIR"
!cp "/content/tweets_with_fasttext_embeddings.csv" "$DRIVE_DIR"

In [26]:
X = df_fasttext.filter(regex=r"^ft_").values

tsne = TSNE(
    n_components=2,
    perplexity=30,
    learning_rate=200,
    max_iter=1000,
    random_state=42,
    verbose=1
)
X2 = tsne.fit_transform(X)

df_fasttext["tsne_x"] = X2[:, 0]
df_fasttext["tsne_y"] = X2[:, 1]

df_plot = df_fasttext.copy()
df_plot["annotation"] = df_plot.get("annotation", "brak").fillna("brak")

fig = px.scatter(
    df_plot,
    x="tsne_x", y="tsne_y",
    color="annotation",
    hover_data={"tweet": True, "annotation": True},
    title="t-SNE wizualizacja embeddingów (FastText)"
)
fig.update_traces(marker=dict(size=6, opacity=0.7))
fig.show()

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 101 samples in 0.001s...
[t-SNE] Computed neighbors for 101 samples in 0.024s...
[t-SNE] Computed conditional probabilities for sample 101 / 101
[t-SNE] Mean sigma: 0.155710
[t-SNE] KL divergence after 250 iterations with early exaggeration: 78.492813
[t-SNE] KL divergence after 1000 iterations: 0.703633


# Word2Vec

In [ ]:
DRIVE_GZ = f"{DRIVE_DIR}/nkjp+wiki-forms-all-300-cbow-hs.txt.gz"

In [27]:
!cp "$DRIVE_GZ" /content/

In [28]:
!gunzip -kf "/content/nkjp+wiki-forms-all-300-cbow-hs.txt.gz"
MODEL_PATH = "/content/nkjp+wiki-forms-all-300-cbow-hs.txt"
VECTOR_SIZE = 300

In [29]:
!pip install gensim

In [30]:
from gensim.models import KeyedVectors

In [31]:
kv = KeyedVectors.load_word2vec_format(MODEL_PATH, binary=False)

In [33]:
print("Wymiar wektorów:", kv.vector_size)
print("Przykładowe słowa:", list(kv.key_to_index)[:5])

Wymiar wektorów: 300
Przykładowe słowa: ['w', 'i', 'na', 'z', 'się']


In [ ]:
tqdm.pandas()

TOKENIZER = re.compile(r"\w+", re.UNICODE) # simple tokenization, alfanumeric words (no emoticons etc)

def normalize(text: str):
    text = re.sub(r"http\S+|www\.\S+", " ", str(text))
    text = re.sub(r"\s+", " ", text).strip()
    return text

def to_tokens(text: str):
    return [w.lower() for w in TOKENIZER.findall(text)]

def sentence_vector_avg(tokens):
    vecs = [kv[w] for w in tokens if w in kv]
    if not vecs:
        return np.zeros(kv.vector_size, dtype=np.float32)
    return np.mean(vecs, axis=0).astype(np.float32)

def embed_row(text):
    text = normalize(text)
    toks = to_tokens(text)
    return sentence_vector_avg(toks)

emb_list = tweets["tweet"].progress_apply(embed_row)
W2V_embeddings = np.vstack(emb_list.values)
W2V_embeddings.shape

100%|██████████| 101/101 [00:00<00:00, 1841.82it/s]


(101, 300)

In [ ]:
np.save("/content/embeddings_w2v_avg.npy", W2V_embeddings)
df_out = pd.concat([tweets.reset_index(drop=True), pd.DataFrame(W2V_embeddings, columns=[f"e{i}" for i in range(W2V_embeddings.shape[1])])], axis=1)
df_out.to_csv("/content/tweets_with_w2v_embeddings.csv", index=False)

In [ ]:
!cp "/content/tweets_with_w2v_embeddings.csv" "$DRIVE_DIR"

In [27]:
DRIVE_CSV = f"{DRIVE_DIR}/tweets_with_w2v_embeddings.csv"
!cp "$DRIVE_CSV" /content/

In [28]:
df_out = pd.read_csv("tweets_with_w2v_embeddings.csv")

In [30]:
embed_cols = [c for c in df_out.columns if c.startswith("e")]
X = df_out[embed_cols].values

tsne = TSNE(
    n_components=2,
    perplexity=20,
    learning_rate=200,
    max_iter=1000,
    random_state=42,
    verbose=1
)
X_2d = tsne.fit_transform(X)

df_out["tsne_x"] = X_2d[:, 0]
df_out["tsne_y"] = X_2d[:, 1]

df_out.head()

[t-SNE] Computing 61 nearest neighbors...
[t-SNE] Indexed 101 samples in 0.001s...
[t-SNE] Computed neighbors for 101 samples in 0.021s...
[t-SNE] Computed conditional probabilities for sample 101 / 101
[t-SNE] Mean sigma: 2.387819
[t-SNE] KL divergence after 250 iterations with early exaggeration: 81.107742
[t-SNE] KL divergence after 1000 iterations: 0.994898


,tweet,annotation,e0,e1,e2,e3,e4,e5,e6,e7,...,e292,e293,e294,e295,e296,e297,e298,e299,tsne_x,tsne_y
0,Nikt nigdy nie rozsiewał takiego smrodu jak,1,0.199123,0.461425,-0.369398,-0.280695,0.129729,0.458937,0.314364,-0.513682,...,0.712148,0.594199,-0.326637,-0.543147,-0.739177,0.003502,-0.422353,0.242517,21.415741,27.387608
1,Nikt cię tak dobrze nie ubierze jak matka chrz...,0,-0.216646,0.334830,-0.648803,0.040806,0.097540,-0.547249,0.162208,-0.293700,...,0.587312,0.385320,-0.745047,-0.330612,-0.248585,0.054231,-0.281700,0.191878,25.428068,25.420055
2,Nikt tutaj nie chce plagiaciarzy kopiujących b...,0,-0.125417,0.606883,-0.222338,-0.053233,0.509792,-0.073351,-0.066285,-0.532081,...,0.033541,0.471372,-0.540420,0.074487,-0.705233,0.362297,-0.021890,-0.409567,3.311413,-15.912837
3,A ksiądz co bierze 12 tyś za spowiedź przez tel?,0,0.163627,0.504219,0.796352,0.320846,0.367560,0.034895,-0.396554,-0.332975,...,0.754752,0.157044,-0.410862,0.106813,0.877793,0.054768,-0.608951,0.627485,29.560791,-9.784841
4,"Koń też ma dużą głowę. Tylko wydaje mi się, że...",1,0.114773,0.143114,0.273522,0.065979,0.731025,-0.455527,0.372835,-0.373813,...,0.601824,0.446618,0.102584,-0.161361,-0.165775,0.234800,-0.126785,-0.164331,4.163102,-5.906681


In [31]:
fig = px.scatter(
    df_out,
    x="tsne_x",
    y="tsne_y",
    color="annotation",  # jeśli masz etykiety; jeśli nie, możesz pominąć
    hover_data={"tweet": True, "annotation": True},
    title="t-SNE wizualizacja embeddingów (Word2Vec)"
)

fig.update_traces(marker=dict(size=6, opacity=0.7))
fig.show()


# TF-IDF

In [ ]:
def normalize(text: str):
    text = re.sub(r"http\S+|www\.\S+", " ", str(text))
    text = re.sub(r"\s+", " ", text).strip()
    return text

tweets_clean = tweets["tweet"].astype(str).map(normalize)
tfidf = TfidfVectorizer(token_pattern=r"\w+", lowercase=True)
X_tfidf = tfidf.fit_transform(tweets_clean)

vocab = tfidf.get_feature_names_out()


In [ ]:
tqdm.pandas()

def embed_row_tfidf(row_idx: int) -> np.ndarray:
    row = X_tfidf[row_idx]        # csr row
    indices = row.indices
    weights = row.data            # tf-idf wagi dla słów w tym tweecie
    vec = np.zeros(kv.vector_size, dtype=np.float32)
    wsum = 0.0
    for idx, w in zip(indices, weights):
        token = vocab[idx]
        if token in kv:
            vec += kv[token] * w
            wsum += w
    if wsum > 0:
        vec /= wsum               # normalizacja średniej ważonej
    return vec

W2V_tfidf = np.vstack([embed_row_tfidf(i) for i in tqdm(range(len(tweets)))])
W2V_tfidf.shape


100%|██████████| 101/101 [00:00<00:00, 1490.49it/s]


(101, 300)

In [ ]:
df_out_tfidf = pd.concat([tweets.reset_index(drop=True), pd.DataFrame(W2V_tfidf, columns=[f"w2v_tfidf_{i}" for i in range(W2V_tfidf.shape[1])])],
    axis=1
)

df_out_tfidf.to_csv("/content/tweets_with_w2v_tfidf.csv", index=False)


In [ ]:
emb_cols_tfidf = [c for c in df_out_tfidf.columns if c.startswith("w2v_tfidf_")]
X = df_out_tfidf[emb_cols_tfidf].values

tsne = TSNE(n_components=2, perplexity=30, learning_rate=200, max_iter=1000, random_state=42, verbose=1)
X2 = tsne.fit_transform(X)

df_out_tfidf["tsne_x"] = X2[:,0]
df_out_tfidf["tsne_y"] = X2[:,1]


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 101 samples in 0.001s...
[t-SNE] Computed neighbors for 101 samples in 0.007s...
[t-SNE] Computed conditional probabilities for sample 101 / 101
[t-SNE] Mean sigma: 2.883304
[t-SNE] KL divergence after 250 iterations with early exaggeration: 92.026382
[t-SNE] KL divergence after 1000 iterations: 0.865293


In [ ]:
df_plot = df_out_tfidf.copy()
if "annotation" not in df_plot.columns:
    df_plot["annotation"] = "brak"

fig = px.scatter(
    df_plot,
    x="tsne_x", y="tsne_y",
    color="annotation",
    hover_data={"tweet": True, "annotation": True},
    title="t-SNE (Word2Vec TF-IDF weighted)"
)
fig.update_traces(marker=dict(size=6, opacity=0.7))
fig.show()


# Sentence transformers

In [32]:
!pip -q install -U sentence-transformers


In [33]:
model = SentenceTransformer("distiluse-base-multilingual-cased-v2")
sentences = tweets["tweet"].astype(str).tolist()

emb_use = model.encode(
    sentences,
    batch_size=64,
    show_progress_bar=True,
    normalize_embeddings=True
)
df_use = pd.concat(
    [tweets.reset_index(drop=True),
     pd.DataFrame(emb_use, columns=[f"use_{i}" for i in range(emb_use.shape[1])])],
    axis=1
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [34]:
X = df_use.filter(regex=r"^use_").values
tsne = TSNE(n_components=2, perplexity=30, learning_rate=200, max_iter=1000, random_state=42, verbose=1)
X2 = tsne.fit_transform(X)
df_use["tsne_x"], df_use["tsne_y"] = X2[:,0], X2[:,1]

df_plot = df_use.copy()
df_plot["annotation"] = df_plot.get("annotation","brak").fillna("brak")
fig = px.scatter(df_plot, x="tsne_x", y="tsne_y", color="annotation",
                 hover_data={"tweet": True, "annotation": True},
                 title="t-SNE (distilUSE multilingual)")
fig.update_traces(marker=dict(size=6, opacity=0.7))
fig.show()

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 101 samples in 0.002s...
[t-SNE] Computed neighbors for 101 samples in 0.033s...
[t-SNE] Computed conditional probabilities for sample 101 / 101
[t-SNE] Mean sigma: 0.373674
[t-SNE] KL divergence after 250 iterations with early exaggeration: 56.836712
[t-SNE] KL divergence after 1000 iterations: 0.625139


# Analiza otrzymanych osadzeń

In [35]:
def cos_sim(a, b):
    return float(np.dot(a, b) / (norm(a)*norm(b) + 1e-9))

def get_emb(df, prefix):
    cols = [c for c in df.columns if c.startswith(prefix)]
    X = df[cols].values
    names = df["tweet"].astype(str).values
    labels = df.get("annotation", pd.Series(["brak"]*len(df))).astype(str).values
    return X, names, labels

models = {
    "Word2Vec_avg":  get_emb(df_out, "e"),
    "FastText":      get_emb(df_fasttext, "ft_"),
    "distilUSE":     get_emb(df_use, "use_"),
}

# Zdania które model uznał za podobne, a różnią się znaczeniem

In [42]:
pd.set_option('display.max_colwidth', None)   # pokaż pełny tekst w kolumnach

In [43]:
def top_cross_label_similar_pairs(X, names, labels, k=10):
    S = cosine_similarity(X)
    n = len(names)
    pairs = []
    for i, j in combinations(range(n), 2):
        if labels[i] != labels[j]:
            pairs.append((S[i, j], i, j))
    pairs.sort(reverse=True)
    rows = []
    for s, i, j in pairs[:k]:
        rows.append({
            "sim": round(float(s), 4),
            "label_i": labels[i], "tweet_i": names[i],
            "label_j": labels[j], "tweet_j": names[j],
        })
    return pd.DataFrame(rows)

for m, (X,names,labels) in models.items():
    print(f"\n=== {m}: top cross-label similar pairs ===")
    display(top_cross_label_similar_pairs(X, names, labels, k=5))


=== Word2Vec_avg: top cross-label similar pairs ===


,sim,label_i,tweet_i,label_j,tweet_j
0,0.7011,0,"Odnoszę się tylko do Twojej sugestii: \""A VAR nie powinien być zawsze wtedy, kiedy jest kontrowersja? \""",1,"Tobie nie dorównam w głupocie. No i co z tego, że sobie tak zapowiedział ? To jest jego zasrany obowiązek."
1,0.6960,1,Nikt nigdy nie rozsiewał takiego smrodu jak,0,Nikt cię tak dobrze nie ubierze jak matka chrzestna dziecino
2,0.6813,0,"RT Nie jestem materialistką, ale chcę mieć tak dużo kasy żebym się w niej topiła mood on",1,"Tobie nie dorównam w głupocie. No i co z tego, że sobie tak zapowiedział ? To jest jego zasrany obowiązek."
3,0.6741,0,"Nie jestem materialistką, ale chcę mieć tak dużo kasy żebym się w niej topiła mood on",1,"Tobie nie dorównam w głupocie. No i co z tego, że sobie tak zapowiedział ? To jest jego zasrany obowiązek."
4,0.6694,1,"Tobie nie dorównam w głupocie. No i co z tego, że sobie tak zapowiedział ? To jest jego zasrany obowiązek.",0,"Dlatego zaznaczyłem, że jest to wątpliwość. Nie wiem, czy w wieku 18 lat się jeszcze rośnie."



=== FastText: top cross-label similar pairs ===


,sim,label_i,tweet_i,label_j,tweet_j
0,0.8612,1,"Przeczytam później, ale jestem ciekawy czy jest tam uwzględnione to że ma w dupie zdanie sztabu medycznego.",0,"Dlatego zaznaczyłem, że jest to wątpliwość. Nie wiem, czy w wieku 18 lat się jeszcze rośnie."
1,0.8373,1,"Przeczytam później, ale jestem ciekawy czy jest tam uwzględnione to że ma w dupie zdanie sztabu medycznego.",0,"RT Nie jestem materialistką, ale chcę mieć tak dużo kasy żebym się w niej topiła mood on"
2,0.8315,1,"Przeczytam później, ale jestem ciekawy czy jest tam uwzględnione to że ma w dupie zdanie sztabu medycznego.",0,"Nie jestem materialistką, ale chcę mieć tak dużo kasy żebym się w niej topiła mood on"
3,0.8161,1,"Przeczytam później, ale jestem ciekawy czy jest tam uwzględnione to że ma w dupie zdanie sztabu medycznego.",0,myślisz że innych celebrytów to obchodzi ? Dużo ludzi ma tam coś za uszami.Takie środowisko.
4,0.8158,0,"W jego przypadku statystyka przebiegniętych kilometrów nie ma znaczenia. Czy gra zajebiście, czy fatalnie, biega tyle samo.",1,"Przeczytam później, ale jestem ciekawy czy jest tam uwzględnione to że ma w dupie zdanie sztabu medycznego."



=== distilUSE: top cross-label similar pairs ===


,sim,label_i,tweet_i,label_j,tweet_j
0,0.5959,1,"Zwycięstwa kogo?, czego? Bandy schizoli nad rozumem?",0,A kto inny ma się bić? Każdy zwyciezca ligi wojewódzkiej gra w barazach.
1,0.5415,0,finały tblk były przyzwoicie sędziowane ?! Aha...,1,Tym w sejmie też? Banda hipokrytów i złodziei.
2,0.4953,1,"RT Generalnie, to mam w dupie powstanie.",0,"RT Nie jestem materialistką, ale chcę mieć tak dużo kasy żebym się w niej topiła mood on"
3,0.4937,1,Nikt nigdy nie rozsiewał takiego smrodu jak,0,Nikt cię tak dobrze nie ubierze jak matka chrzestna dziecino
4,0.4647,0,"Tak, to nawet lepsze rozwiązanie.",1,I słusznie. Zakompleksieni malkontenci nie muszą oglądać😁😀😀


# Zdania semantycznie podobne, które model uznał za odległe

In [44]:
def worst_within_label_pairs(X, names, labels, label_value="obraźliwe", k=10):
    idx = np.where(labels == label_value)[0]
    if len(idx) < 2:
        return pd.DataFrame([{"info": f"Za mało próbek dla etykiety {label_value}"}])
    S = cosine_similarity(X[idx])
    pairs = []
    for a, b in combinations(range(len(idx)), 2):
        i, j = idx[a], idx[b]
        pairs.append((S[a, b], i, j))
    pairs.sort()  # rosnąco -> najgorsze pierwsze
    rows = []
    for s, i, j in pairs[:k]:
        rows.append({"sim": round(float(s), 4), "tweet_i": names[i], "tweet_j": names[j]})
    return pd.DataFrame(rows)

target_labels = list(pd.Series(models["Word2Vec_avg"][2]).unique())
for m, (X,names,labels) in models.items():
    print(f"\n=== {m}: within-label (najniższe podobieństwa) ===")
    for L in target_labels[:3]:  # max 3 etykiety dla czytelności
        print(f"\n-- Etykieta: {L}")
        display(worst_within_label_pairs(X, names, labels, label_value=L, k=5))



=== Word2Vec_avg: within-label (najniższe podobieństwa) ===

-- Etykieta: 1


,sim,tweet_i,tweet_j
0,0.0710,Proponuję pozbawić obywatelstwa polskiego i objąć zakazem powrotu do Polski wszystkie te euromendy jak im się skończy kadencja w UE,"Zwycięstwa kogo?, czego? Bandy schizoli nad rozumem?"
1,0.0714,"Generalnie, to mam w dupie powstanie.","Zwycięstwa kogo?, czego? Bandy schizoli nad rozumem?"
2,0.0951,"RT Generalnie, to mam w dupie powstanie.","Zwycięstwa kogo?, czego? Bandy schizoli nad rozumem?"
3,0.1128,Dzisiaj zajmujemy się WARiatem Krystyną ciekawe czy będzie płakał?,Tym w sejmie też? Banda hipokrytów i złodziei.
4,0.1519,Dzisiaj zajmujemy się WARiatem Krystyną ciekawe czy będzie płakał?,Proponuję pozbawić obywatelstwa polskiego i objąć zakazem powrotu do Polski wszystkie te euromendy jak im się skończy kadencja w UE



-- Etykieta: 0


,sim,tweet_i,tweet_j
0,-0.1926,RT Prezes Dunin złożył wypowiedzenie.,myślisz że innych celebrytów to obchodzi ? Dużo ludzi ma tam coś za uszami.Takie środowisko.
1,-0.1924,". \""Napisałbym gdzieś, że mi dobrze bez mediów społecznościowych, ale nie mam gdzie\"". \nKazał Was pozdrowić ;-)",RT Prezes Dunin złożył wypowiedzenie.
2,-0.1793,Ale to wciąż bardziej siła niż technika.,RT Prezes Dunin złożył wypowiedzenie.
3,-0.1788,"Mów prawdę, kto was skrzywdził",➡ #FF dla nowego na TT z KPRP
4,-0.1774,"W jego przypadku statystyka przebiegniętych kilometrów nie ma znaczenia. Czy gra zajebiście, czy fatalnie, biega tyle samo.",RT Prezes Dunin złożył wypowiedzenie.



=== FastText: within-label (najniższe podobieństwa) ===

-- Etykieta: 1


...

,sim,tweet_i,tweet_j
0,0.1469,Ta kadra to rocznik 2002,Rozwiązanie na casual friday. \nModel: Tenisówek wizytowy.
1,0.1624,jestem tu od 2013 xd,Rozwiązanie na casual friday. \nModel: Tenisówek wizytowy.
2,0.1694,finały tblk były przyzwoicie sędziowane ?! Aha...,jestem tu od 2013 xd
3,0.1878,Kończyłeś już kilka razy 😂,➡ #FF dla nowego na TT z KPRP
4,0.1925,Nikt cię tak dobrze nie ubierze jak matka chrzestna dziecino,➡ #FF dla nowego na TT z KPRP



=== distilUSE: within-label (najniższe podobieństwa) ===

-- Etykieta: 1


,sim,tweet_i,tweet_j
0,-0.0994,Nikt nigdy nie rozsiewał takiego smrodu jak,"Przeczytam później, ale jestem ciekawy czy jest tam uwzględnione to że ma w dupie zdanie sztabu medycznego."
1,-0.0443,Proponuję pozbawić obywatelstwa polskiego i objąć zakazem powrotu do Polski wszystkie te euromendy jak im się skończy kadencja w UE,"Mentalnościowo murzyn do kwadratu, dlatego taki czarny"
2,-0.0431,"Z sądu korzystał co 5 Polak z tego połowa jest zadowolona. Skąd niechęć do sadow, z pisdzielskie szczucia, podobnie jak do uchodźców","Generalnie, to mam w dupie powstanie."
3,-0.0291,"Zjeby idą na starcie, żadnych ustępstw, żadnych wątpliwości","Przeczytam później, ale jestem ciekawy czy jest tam uwzględnione to że ma w dupie zdanie sztabu medycznego."
4,-0.0260,Dzisiaj zajmujemy się WARiatem Krystyną ciekawe czy będzie płakał?,Proponuję pozbawić obywatelstwa polskiego i objąć zakazem powrotu do Polski wszystkie te euromendy jak im się skończy kadencja w UE



-- Etykieta: 0


,sim,tweet_i,tweet_j
0,-0.1615,Tak się poznałam z moim chłopakiem 😂 czasami warto,"A VAR nie miał poprawić jakości sędziowania, tylko efekt końcowy - mniej wypaczonych wyników, mniej skandali."
1,-0.1473,Nikt tutaj nie chce plagiaciarzy kopiujących brytyjskich pisarzy.,Skąd ja to znam xd a co się dzieje
2,-0.1370,Nikt cię tak dobrze nie ubierze jak matka chrzestna dziecino,A ile miało być ?
3,-0.1354,"Mów prawdę, kto was skrzywdził","Jeszcze raz to napiszę, dopuki inspekcja pracy nie doprowadzi do przestrzegania czasu pracy niedziela ma być wolna"
4,-0.1293,A ile miało być ?,"\""Zadzwońcie do rady ministrów, że zaraz tam jadę\"""


In [45]:
TOKENIZER = re.compile(r"\w+", re.UNICODE)
def length_tokens(s): return len(TOKENIZER.findall(str(s)))
def length_chars(s):  return len(str(s))

tweets_len = pd.DataFrame({
    "tweet": models["Word2Vec_avg"][1],
    "len_tok": [length_tokens(s) for s in models["Word2Vec_avg"][1]],
    "len_chr": [length_chars(s)  for s in models["Word2Vec_avg"][1]],
})

def length_gap_for_similar_pairs(X, names, sim_thr=0.6, k=2000):
    S = cosine_similarity(X)
    rows = []
    cnt = 0
    for i, j in combinations(range(len(names)), 2):
        if S[i, j] >= sim_thr:
            lt_i = length_tokens(names[i]); lt_j = length_tokens(names[j])
            rows.append({"sim": float(S[i,j]), "len_diff_tokens": abs(lt_i-lt_j)})
            cnt += 1
            if cnt >= k: break
    if not rows:
        return pd.DataFrame([{"info": f"brak par z sim >= {sim_thr}"}])
    df = pd.DataFrame(rows)
    return df["len_diff_tokens"].describe()

for m, (X,names,labels) in models.items():
    print(f"\n=== {m}: różnica długości (tokens) w parach podobnych (cos>=0.6) ===")
    display(length_gap_for_similar_pairs(X, names, sim_thr=0.6))



=== Word2Vec_avg: różnica długości (tokens) w parach podobnych (cos>=0.6) ===


,len_diff_tokens
count,87.000000
mean,4.494253
std,4.283211
min,0.000000
25%,1.000000
50%,3.000000
75%,8.000000
max,15.000000



=== FastText: różnica długości (tokens) w parach podobnych (cos>=0.6) ===


,len_diff_tokens
count,1824.000000
mean,6.004934
std,3.948027
min,0.000000
25%,3.000000
50%,6.000000
75%,9.000000
max,17.000000



=== distilUSE: różnica długości (tokens) w parach podobnych (cos>=0.6) ===


,len_diff_tokens
count,5.000000
mean,0.800000
std,0.447214
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


korelacja między długością a normą wektora

In [46]:
def norm_vs_length(X, names):
    norms = norm(X, axis=1)
    lens = np.array([length_tokens(s) for s in names])
    corr = np.corrcoef(norms, lens)[0,1]
    return corr, norms.mean(), norms.std()

for m, (X,names,labels) in models.items():
    corr, mu, sd = norm_vs_length(X, names)
    print(f"{m}: corr(len_tokens, ||vec||)={corr:.3f} | norm mean={mu:.3f} ± {sd:.3f}")


Word2Vec_avg: corr(len_tokens, ||vec||)=-0.691 | norm mean=7.506 ± 1.574
FastText: corr(len_tokens, ||vec||)=-0.618 | norm mean=0.520 ± 0.066
distilUSE: corr(len_tokens, ||vec||)=-0.007 | norm mean=1.000 ± 0.000


# Czy opracowane osadzenia nadają się do klasyfikacji tweetów na obraźliwe i nieobraźliwe?

In [41]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression

def quick_cv_score(X, y, cv=5):
    clf = LogisticRegression(max_iter=2000, class_weight="balanced")
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)
    f1 = cross_val_score(clf, X, y, scoring="f1_macro", cv=skf)
    return f1.mean(), f1.std()

y_raw = tweets["annotation"].astype(str).values
le = LabelEncoder()
y = le.fit_transform(y_raw)

for m, (X,names,labels) in models.items():
    mask = pd.Series(labels).notna().values
    Xm, ym = X[mask], y[mask]
    mean_f1, std_f1 = quick_cv_score(Xm, ym, cv=5)
    print(f"{m}: F1-macro (5CV) = {mean_f1:.3f} ± {std_f1:.3f}")

Word2Vec_avg: F1-macro (5CV) = 0.628 ± 0.116
FastText: F1-macro (5CV) = 0.611 ± 0.056
distilUSE: F1-macro (5CV) = 0.580 ± 0.160
